# LangChain Basics

In [1]:
# installing the required libraries
#!pip install -r ./requirements.txt -q

In [2]:
# !pip - installs the packages in the base environment
# pip - installs the packages in the virtual environment

In [3]:
pip show langchain

Name: langchain
Version: 0.0.271
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: c:\users\aadhil_105729\appdata\local\anaconda3\envs\ml\lib\site-packages
Requires: async-timeout, pydantic, requests, SQLAlchemy, numexpr, PyYAML, aiohttp, langsmith, dataclasses-json, numpy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
# upgrading langchain
! pip install langchain --upgrade -q

In [5]:
#!pip install python-dotenv

#### Python-dotenv

In [6]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (OpenAI, Pinecone) from .env
load_dotenv(find_dotenv(), override=True)

os.environ.get('PINECONE_API_KEY')

'7970d625-c370-4317-8d76-26b7e1433bb9'

### LLM Models (Wrappers): GPT-3

In [7]:
from langchain.llms import OpenAI

In [8]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

In [9]:
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [10]:
output = llm('explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is the branch of physics that describes the behavior of matter and energy at the subatomic scale.


In [11]:
print(llm.get_num_tokens('explain quantum mechanics in one sentence'))

7


In [12]:
output = llm.generate(['... is the capital of France.',
                   'What is the formula for the area of a circle?'])

In [13]:
print(output.generations)

[[Generation(text='\n\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where A is the area and r is the radius.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [14]:
print(output.generations[0][0].text)



Paris


In [15]:
len(output.generations)

2

In [18]:
output = llm.generate(['Write an orignal tagline for a burger restaurant'] * 3)

In [19]:
for o in output.generations:
    print(o[0].text, end='')



"Taste the Difference - Our Burgers are Freshly Grilled!"

"A Burger So Good, You'd Swipe Right!"

"Sink your teeth into a delicious burger experience!"

### ChatModels: GPT-3.5-Turbo and GPT-4

In [20]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [21]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [22]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch mathematische Modelle und ermöglicht Vorhersagen über Wahrscheinlichkeiten von Ereignissen.


### Prompt Templates

In [23]:
from langchain import PromptTemplate

In [24]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] output_parser=None partial_variables={} template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.' template_format='f-string' validate_template=True


In [27]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='HIV', language='English'))
print(output)



HIV, or Human Immunodeficiency Virus, is a virus that affects the body's immune system, making it unable to fight off infections and diseases. HIV is transmitted through contact with infected bodily fluids, including blood, semen, vaginal fluid, and breast milk. HIV is a serious and life-threatening condition, and there is no cure. Treatment with antiretroviral drugs can help people manage the virus, but it cannot be completely eliminated.


### Simple Chains

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'french'})


In [29]:
print(output)

Le HSV, également connu sous le nom de virus de l'herpès simplex, est un virus à ADN de la famille des Herpesviridae. Il existe deux types principaux de HSV : le HSV-1, qui provoque généralement des lésions buccales et labiales, et le HSV-2, qui est principalement associé aux lésions génitales. Le HSV peut être transmis par contact direct avec une personne infectée ou par le biais de rapports sexuels non protégés. Bien qu'il ne puisse pas être guéri, il peut être traité avec des médicaments antiviraux pour réduire les symptômes et les récidives.


### Sequential Chains

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')





> Entering new SimpleSequentialChain chain...


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
The function `softmax(x)` takes in an input array `x` and calculates the softmax values for each set of scores in `x`.

The softmax function is commonly used in machine learning for problems involving multi-class classification. It converts a vector of real numbers into a probability distribution over the classes, with the requirements that the probabilities sum up to 1 and each probability is between 0 and 1.

The steps in the function are as follows:
1. The exponential function np.exp(x) is applied element-wise to the input array `x`. This is done to transform the scores into positive values, giving more weight to larger scores.
2. The sum of each set of exponential scores is calculated along the axis 0 using np.sum(np.exp(x), axis=0). This sum is used in the next step for normalization.
3. Each exponential s

### LangChain Agents

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
# agent_executor.run('Calculate the square root of the factorial of 20 \
# and display it with 4 decimal points')

agent_executor.run('what is the answer to 5.1 ** 7.3?')

